In [3]:
# load ASE atom from POSCAR file
from ase.io.vasp import read_vasp
import matplotlib.pyplot as plt
import os
import numpy as np
from glob import glob
import pickle

from ase import Atoms
from ase.neighborlist import NeighborList
from ase.build import make_supercell

from ovito.modifiers import CoordinationAnalysisModifier
from ovito.pipeline import StaticSource, Pipeline
from ovito.io.ase import ase_to_ovito

from mlptools.data.structures import generate_crystal
from mlptools.io.write import QuantumEspressoWriter

In [19]:
# randomly displace atoms while compressing or expanding the cell
path2root = '/Users/y1u0d2/Google Drive/マイドライブ/HamaLab_kotani/QE/SiO2/stishovite__mp-6947'
path2POSCAR = os.path.join(path2root, "original", "POSCAR")
path2save = os.path.join(path2root, "relax")
path2input = os.path.join(path2root, "input")

original_atoms = read_vasp(path2POSCAR)

supercell_atoms = make_supercell(original_atoms, [[2,0,0],[0,2,0],[0,0,2]])

print(f"Number of atoms in original: {len(original_atoms)}")
print(f"Number of atoms in supercell: {len(supercell_atoms)}")

Number of atoms in original: 6
Number of atoms in supercell: 48


In [17]:
with open(os.path.join(path2input, "relax.in")) as f:
    original_scf_input_lines = [s.strip() for s in f.readlines()]

for i, atoms in enumerate([supercell_atoms]):
    writer = QuantumEspressoWriter(atoms, path2input, scf_filename="relax.in")
    scf_input_lines = writer.output()

    # create dir
    path2result = f'{path2save}/relax_{i}'
    if not os.path.exists(path2result):
        os.makedirs(path2result)
    else:
        print(f'{path2result} already exists')

    # write scf.in
    with open(f'{path2result}/relax.in', 'w') as f:
        f.writelines('\n'.join(scf_input_lines))

# 計算完了済みのSCFを初期構造にして構造最適化計算を行う準備

In [4]:
# SiO2
# path2scf = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-546794/result"
# path2relax = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-546794/relax"
# path2input = "/Users/y1u0d2/desktop/Lab/result/qe/SiO2/mp-546794/input"
# path2outdir_root = "/home/y1u0d2/result/qe/SiO2/mp-546794/relax"

# num_samples = 350
# all_scf_dirs = glob(f"{path2scf}/scf_*")
# print(f"Number of scf dirs: {len(all_scf_dirs)}")

# sampled_scf_dirs = np.random.choice(all_scf_dirs, num_samples)

Number of scf dirs: 7697


In [6]:
# Si
from mlptools.db.db import get_session
from mlptools.db.model import Structure
from glob import glob
import pickle
import os

ATOMIC_SYMBOL = "Si"
PATH2ATOMS = "/Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms"

session = get_session(structure_symbol=ATOMIC_SYMBOL)
all_structure = session.query(Structure).filter(Structure.structure_name == "amorphous").all()

all_target_scf_dir = []
for structure in all_structure:
    path2target = os.path.join(PATH2ATOMS, structure.id, "atoms.pkl")
    if not os.path.exists(path2target):
        print(f"atoms.pkl not fount in {path2target}")
    atoms = pickle.load(open(path2target, "rb"))
    if atoms.n_atoms < 0:
        continue
    else:
        all_target_scf_dir.append(os.path.join(PATH2ATOMS, structure.id))
        print(f"[ADD] {path2target}")

[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/d46049a7-a5ba-428c-a189-f8bd2074880a/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/b7f1aaed-d939-43e4-ba92-155d45b55e3d/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/a0faf8f3-6908-45a2-9e07-f91d471b380b/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/481f6333-a4c8-4834-8a19-147deeebbdb8/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/74efc4c9-2574-42ea-a172-f3e8cf4a385e/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/2e85e94c-0e7b-4101-b488-d5ee4d3114a8/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/e5970640-ed94-4db3-9052-e08959331a67/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/d63c1a34-9f4d-483f-b6f3-e7b1798edd9e/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/3bc017ff-341c-4b2c-b3eb-43d0a37ab18e/atoms.pkl
[ADD] /Users/y1u0d2/desktop/Lab/data/qe_data/Si/atoms/abbf7f1f-8f22-433a-be12-dbc1d1407207/

In [7]:
path2relax = "/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/relax/result"
path2input = "/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/template"
path2outdir_root = "/home/y1u0d2/result/qe/Si/amorphous/relax/result"

num_samples = 1000

sampled_scf_dirs = np.random.choice(all_target_scf_dir, num_samples)

In [8]:
# 全原子取得
for scf_dir in sampled_scf_dirs:
    atoms = pickle.load(open(f"{scf_dir}/atoms.pkl", "rb"))

    # create dir
    basename = os.path.basename(scf_dir)
    new_basename = f"relax_{basename}"
    path2result = f'{path2relax}/{new_basename}'
    if not os.path.exists(path2result):
        os.makedirs(path2result)
    else:
        print(f'{path2result} already exists')
    
    # get scf lines
    path2outdir = os.path.join(path2outdir_root, new_basename)
    writer = QuantumEspressoWriter(atoms.ase_atoms, path2input, scf_filename="relax.in", out_dir=path2outdir)
    scf_input_lines = writer.output()

    # write scf.in
    with open(f'{path2result}/relax.in', 'w') as f:
        f.writelines('\n'.join(scf_input_lines))

/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/relax/result/relax_a4187211-0e21-4208-bc0d-1cc76ae9e0a5 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/relax/result/relax_6738448d-fd31-4601-885b-89198b344193 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/relax/result/relax_73232400-da23-4839-8406-bc6dc8c93659 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/relax/result/relax_6a280de6-0625-4504-aa90-b580c701b326 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/relax/result/relax_f7ac5387-6f0d-4831-a74b-0943af666ee0 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/relax/result/relax_ecc87068-4a10-4d83-80f2-21123f597406 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/relax/result/relax_9c5129fd-3dd5-4391-b552-fc8c709906b1 already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/amorphous/relax/result/relax_a47aaece-36fc-4aad-b150-a933a099084d already exists
/Users/y1u0d2/desktop/Lab/result/qe/Si/a